In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd drive/My\ Drive/TryTextAttackAgain

/content/drive/My Drive/TryTextAttackAgain


In [3]:
!ls 

BaseLoading.ipynb				  multi_nli_train_word_freq.csv
corpus.py					  preprocess_datasets
DefendBertGuide.ipynb				  prioritized.py
detect_adversarial.py				  __pycache__
EvalBert.ipynb					  RunFull
freq_attacks_mnli.py				  snli_train_word_freq.csv
freq_attacks_snli.py				  TextAttack
freq_stats.py					  TrainedBertModels
InferSent.ipynb					  TrainNotebooks
inferSentModel.py				  wandb
InferSent_snli_pretrained_GLOVEepoch10.fullmodel


In [4]:
#!git clone https://github.com/QData/TextAttack.git

In [5]:

%cd TextAttack/

/content/drive/My Drive/TryTextAttackAgain/TextAttack


In [6]:
!pip install pyarrow>=0.16.0 --force-reinstall
!python3 setup.py install

ERROR: tensorflow 2.3.0 has requirement numpy<1.19.0,>=1.16.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
ERROR: textattack 0.2.4 has requirement numpy<1.19.0, but you'll have numpy 1.19.1 which is incompatible.
ERROR: flair 0.6 has requirement pytest>=5.3.2, but you'll have pytest 3.6.4 which is incompatible.
running install
running bdist_egg
running egg_info
writing textattack.egg-info/PKG-INFO
writing dependency_links to textattack.egg-info/dependency_links.txt
writing entry points to textattack.egg-info/entry_points.txt
writing requirements to textattack.egg-info/requires.txt
writing top-level names to textattack.egg-info/top_level.txt
writing manifest file 'textattack.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
run

In [7]:
!pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.9.0          
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         19.3.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bert-score                    0.3.5          
bleach                        3.1.5          
blis                          0.4.

In [8]:
!ls ../

BaseLoading.ipynb				  multi_nli_train_word_freq.csv
corpus.py					  preprocess_datasets
DefendBertGuide.ipynb				  prioritized.py
detect_adversarial.py				  __pycache__
EvalBert.ipynb					  RunFull
freq_attacks_mnli.py				  snli_train_word_freq.csv
freq_attacks_snli.py				  TextAttack
freq_stats.py					  TrainedBertModels
InferSent.ipynb					  TrainNotebooks
inferSentModel.py				  wandb
InferSent_snli_pretrained_GLOVEepoch10.fullmodel


In [9]:
import textattack

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [10]:
!ls ../TrainedBertModels/roberta-base-snli

1598279886.7127306				   pytorch_model.bin
attack						   README.md
config.json					   special_tokens_map.json
events.out.tfevents.1598279886.3762d14d0a9a.368.0  tokenizer_config.json
log.txt						   train_args.json
merges.txt					   vocab.json


# Evaluating model

In [ ]:
!textattack eval --model-from-huggingface  ../TrainedBertModels/roberta-base-snli  --dataset-from-nlp snli --num-examples 10000 --dataset_split test --shuffle False

2020-08-25 07:11:39.645012: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli
textattack: Loading nlp dataset snli, split test.
textattack: Got 10000 predictions.
textattack: Successes 9003/10000 (90.03%)


# Create frequency dictionary

In [11]:
import string
import nltk
import nlp
import csv
import os
import math 
from textattack.shared.utils import words_from_text
import numpy as np

def get_tokenize_sentences(premises,hypothesis):
  all_sentences = []

  for i in zip(premises,hypothesis):
    st = i[0] + " " + i[1]
    tokens = words_from_text(st)
    all_sentences.append(tokens)
  
  return all_sentences

def return_multi_nli_train_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  train_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))
  train_indices = np.delete(np.arange(len(train_data)),eval_indices)

  train_dict['premise'] = np.array(train_data['premise'])[train_indices] 
  train_dict['hypothesis'] = np.array(train_data['hypothesis'])[train_indices]

  return train_dict

def create_freq_corpus(dataset,csvname):


  if dataset is 'multi_nli':
   print('MULTI') 
   train_data = return_multi_nli_train_dict(dataset)

  else:

    train_data = nlp.load_dataset(dataset)['train']

  
  all_sentences = get_tokenize_sentences(train_data['premise'],train_data['hypothesis'])

  freq_dict = {}

  for sentence in all_sentences:
    for token in sentence:
      if token  in freq_dict:
        freq_dict[token] += 1
      else:
        freq_dict[token] = 1 

  print("WRITE")
  with open(csvname, 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in freq_dict.items():
      writer.writerow([key, value])
  

  return freq_dict

def load_csv(filename):

  with open(filename) as csv_file:
    reader = csv.reader(csv_file)
    freq_dict = dict(reader)
  
  return freq_dict


# SNLI

In [12]:
snli_file_name = '../snli_train_word_freq.csv'
snli_dataset_name = 'snli'
#snli_freq_dict = create_freq_corpus(snli_dataset_name,snli_file_name)
snli_freq_dict = load_csv(snli_file_name)

In [13]:
snli_freq_dict['the']

'370390'

# To get frequency statistics

In [14]:
import pandas as pd
from ast import literal_eval
import numpy as np
import math 

def phi_value(freq):
  return math.log(1+freq)

def get_phi_values(words,freq_dict,in_dict=False):

  if in_dict:
    return [phi_value(int(freq_dict[word])) for word in words if word in freq_dict]

  return [phi_value(int(freq_dict[word])) if word in freq_dict else 0 for word in words]


def get_attack_freq_stats(filename,freq_dict):
  
  attack_data = pd.read_csv(filename)

  premises = [i['premise'] for _, i in attack_data.iterrows()] 
  attacked_hypo = [i['attacked_hypothesis'] for _, i in attack_data.iterrows()] 
  original_hypo = [i['original_hypothesis'] for _, i in attack_data.iterrows()] 
  modifiable_indices = [literal_eval(i['modifiable_indices']) for _, i in attack_data.iterrows()] 
  modified_indices = [literal_eval(i['modified_indices']) for _, i in attack_data.iterrows()] 

  attacked_sentences = get_tokenize_sentences(premises,attacked_hypo)
  original_sentences = get_tokenize_sentences(premises,original_hypo)
  
  replaceable_words =[]
  replaced_words = []
  sub_words = []

  for i in range(len(original_sentences)):
    replaceable_words.append(np.array(original_sentences[i])[modifiable_indices[i]])
    replaced_words.append(np.array(original_sentences[i])[modified_indices[i]])
    sub_words.append(np.array(attacked_sentences[i])[modified_indices[i]])

  replaceable_words_freqs = []
  replaced_words_freqs = []
  sub_words_freqs = []
  sub_ovv_freqs = []

  for i in range(len(replaceable_words)):
    replaceable_words_freqs = np.concatenate((replaceable_words_freqs,get_phi_values(replaceable_words[i],freq_dict)))
    replaced_words_freqs = np.concatenate((replaced_words_freqs,get_phi_values(replaced_words[i],freq_dict)))
    sub_words_freqs = np.concatenate((sub_words_freqs,get_phi_values(sub_words[i],freq_dict)))
    sub_ovv_freqs = np.concatenate((sub_ovv_freqs,get_phi_values(sub_words[i],freq_dict, True)))

  print("MEAN:\t" + str(np.mean(replaceable_words_freqs)) + "\t STD:\t" + str(np.std(replaceable_words_freqs)))
  print("MEAN:\t" + str(np.mean(replaced_words_freqs)) + "\t STD:\t" + str(np.std(replaced_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_words_freqs)) + "\t STD:\t" + str(np.std(sub_words_freqs)))
  print("MEAN:\t" + str(np.mean(sub_ovv_freqs)) + "\t STD:\t" + str(np.std(sub_ovv_freqs)))

In [15]:
!ls ../TrainedBertModels/roberta-base-snli

1598279886.7127306				   pytorch_model.bin
attack						   README.md
config.json					   special_tokens_map.json
events.out.tfevents.1598279886.3762d14d0a9a.368.0  tokenizer_config.json
log.txt						   train_args.json
merges.txt					   vocab.json


# Attacks

In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli --recipe random_nli  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.


[Succeeded / Failed / Total] 423 / 806 / 1371:  69% 1371/2000 [01:01<00:28, 22.43it/s]--------------------------------------------- Result 1372 ---------------------------------------------
Neutral (60%) --> [FAILED]

Premise: A hockey player in helmet.
Hypothesis: They are playing hockey


[Succeeded / Failed / Total] 423 / 807 / 1372:  69% 1372/2000 [01:01<00:27, 22.44it/s]--------------------------------------------- Result 1373 ---------------------------------------------
Neutral (99%) --> [FAILED]

Premise: Bespectacled man in suit opens wallet while a younger lady in formal wear laughs with him.
Hypothesis: A man shows his girlfriend the contents of his wallet.


[Succeeded / Failed / Total] 423 / 808 / 1373:  69% 1373/2000 [01:01<00:27, 22.43it/s]--------------------------------------------- Result 1374 ---------------------------------------------
Contradiction (100%) --> [FAILED]

Premise: She is standing outside a building

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/random_nli_2000_attacked.csv',snli_freq_dict)

MEAN:	10.164118101802039	 STD:	2.7852977263496927
MEAN:	10.129686076577654	 STD:	2.7612617144013067
MEAN:	3.250908350729607	 STD:	3.523602046409696
MEAN:	5.209566144930333	 STD:	3.1132717478357574


In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli --attack-from-class ../prioritized Priotirized  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.
[Succeeded / Failed / Total] 685 / 577 / 1408:  70% 1408/2000 [03:08<01:19,  7.48it/s]--------------------------------------------- Result 1409 ---------------------------------------------
Neutral (83%) --> Contradiction (77%)

Premise: A male baseball player for the baseball team The Indians prepares to catch a ball.
Hypothesis: That player has never missed a catch.

Premise: A male baseball player for the baseball team The Indians prepares to catch a ball.
Hypothesis: That player has never pretermit a catch.


[Succeeded / Failed / Total] 686 / 577 / 1409:  70% 1409/2000 [03:08<01:19,  7.48it/s]--------------------------------------------- Result 1410 ---------------------------------------------
Entailment (99%) --> Neutral (87%)

Premise: A man in an Alaska sweatshirt stands behind a counter.
Hypothesis: The person behind the counter is a man.

Premise: A man in an Alaska sweatshirt stands behind a counter.
Hypothesis: The person 

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/prioritized_Priotirized_2000_attacked.csv',snli_freq_dict)

MEAN:	10.164118101802039	 STD:	2.7852977263496927
MEAN:	10.123931820479712	 STD:	2.646631287306249
MEAN:	2.857579416245049	 STD:	3.2468792502719186
MEAN:	4.874360117571078	 STD:	2.855178992356303


In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli --recipe textfooler  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.

[Succeeded / Failed / Total] 1277 / 54 / 1481:  74% 1481/2000 [12:02<04:13,  2.05it/s]--------------------------------------------- Result 1482 ---------------------------------------------
Neutral (99%) --> Entailment (62%)

Premise: Three black people, two men and one woman, in the middle of a small, rural street, are piling up items on top of a taxi cab with a net dangling down on top of the windshield, while two children and two other people are nearby.
Hypothesis: The people put furniture on the taxi

Premise: Three black people, two men and one woman, in the middle of a small, rural street, are piling up items on top of a taxi cab with a net dangling down on top of the windshield, while two children and two other people are nearby.
Hypothesis: The people laid furnishing on the taxi


[Succeeded / Failed / Total] 1278 / 54 / 1482:  74% 1482/2000 [12:03<04:12,  2.05it/s]--------------------------------------------- Result 1483 ---

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/textfooler_2000_attacked.csv',snli_freq_dict)

MEAN:	9.054576312926118	 STD:	2.730533669168767
MEAN:	8.460748024397974	 STD:	2.5215698946800233
MEAN:	2.704001681757439	 STD:	3.0583232638995774
MEAN:	4.393426268597362	 STD:	2.7883369661982167


In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli --recipe pwws_snli  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.
Hypothesis: A guy stands next to a trashcan.

Premise: A man is standing outside next to a garbage can.
Hypothesis: A cat stands next to a trashcan.


[Succeeded / Failed / Total] 1178 / 144 / 1471:  74% 1471/2000 [14:01<05:02,  1.75it/s]--------------------------------------------- Result 1472 ---------------------------------------------
Entailment (98%) --> Neutral (93%)

Premise: A gymnastics performer is in mid-pose as she completes her jump.
Hypothesis: A gymnast is completing her jump.

Premise: A gymnastics performer is in mid-pose as she completes her jump.
Hypothesis: A gymnast is completing her parachuting.


[Succeeded / Failed / Total] 1179 / 144 / 1472:  74% 1472/2000 [14:02<05:02,  1.75it/s]--------------------------------------------- Result 1473 ---------------------------------------------
Entailment (74%) --> Neutral (98%)

Premise: A woman in black reviews a message as she walks to work.
Hypothesis: The woman in bla

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/pwws_snli_2000_attacked.csv',snli_freq_dict)

MEAN:	9.049272434260248	 STD:	2.7331396093253293
MEAN:	8.866279601814144	 STD:	2.436121323442647
MEAN:	2.9542328864897986	 STD:	3.213134388646241
MEAN:	4.767756195999721	 STD:	2.8311767009314344


In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli --recipe alzantot_gpt2  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.
Premise: A boy whiffs a ball with a bat.
Hypothesis: The boy is playing baseball.

Premise: A boy whiffs a ball with a bat.
Hypothesis: The dude is toying baseball.


[Succeeded / Failed / Total] 969 / 327 / 1445:  72% 1445/2000 [1:38:21<37:46,  4.08s/it]--------------------------------------------- Result 1446 ---------------------------------------------
Entailment (98%) --> Contradiction (100%)

Premise: This woman is skiing down a snowy hill.
Hypothesis: This woman is on a hill.

Premise: This woman is skiing down a snowy hill.
Hypothesis: This woman is on a colina.


[Succeeded / Failed / Total] 970 / 327 / 1446:  72% 1446/2000 [1:38:22<37:41,  4.08s/it]--------------------------------------------- Result 1447 ---------------------------------------------
Contradiction (100%) --> [FAILED]

Premise: Villagers pulling in the fish boat.
Hypothesis: The villagers are sitting around the camp fire.


[Succeeded / Failed / Total] 970 / 3

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/alzantot_gpt2_2000_attacked.csv',snli_freq_dict)

MEAN:	9.049272434260248	 STD:	2.7331396093253293
MEAN:	8.720714627998039	 STD:	2.3959275255407215
MEAN:	3.271626050188479	 STD:	3.395912310318605
MEAN:	4.824870605106053	 STD:	3.084328026242011


# Frequency Attacks

In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli/ --attack-from-class ../freq_attacks_snli FrequencyR  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.
Premise: Two Asian people cooking on a grill in a park.
Hypothesis: Seventh are hot andiron ja hamburgs sobre nova grillwork.


[Succeeded / Failed / Total] 911 / 424 / 1486:  74% 1486/2000 [02:18<00:47, 10.73it/s]--------------------------------------------- Result 1487 ---------------------------------------------
Entailment (82%) --> [SKIPPED]

Premise: Two workers in work attire (windbreakers and vests) walking along the edge of a road.
Hypothesis: The workers were wearing safty gear.


[Succeeded / Failed / Total] 911 / 424 / 1487:  74% 1487/2000 [02:18<00:47, 10.74it/s]Try loading SNLI training Corpus
--------------------------------------------- Result 1488 ---------------------------------------------
Neutral (100%) --> Contradiction (100%)

Premise: A gymnastics performer is in mid-pose as she completes her jump.
Hypothesis: A gymnast is completing her jump in the finals.

Premise: A gymnastics performer is in mid-pose as she 

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/freq_attacks_snli_FrequencyR_2000_attacked.csv',snli_freq_dict)

MEAN:	10.164118101802039	 STD:	2.7852977263496927
MEAN:	10.173224933829538	 STD:	2.7283111511048377
MEAN:	0.10760951613426399	 STD:	0.595203042940518
MEAN:	2.18046727583005	 STD:	1.6305295776857915


In [ ]:
!textattack attack --attack_output_snli ../TrainedBertModels/roberta-base-snli --attack-from-class ../freq_attacks_snli FrequencyP  --num-examples 2000 --dataset-from-nlp snli --model-from-huggingface ../TrainedBertModels/roberta-base-snli

Streaming output truncated to the last 5000 lines.

Premise: A variety of people are sitting around a tale doing craft projects.
Hypothesis: The people are creative.


[Succeeded / Failed / Total] 937 / 399 / 1489:  74% 1489/2000 [02:17<00:47, 10.85it/s]Try loading SNLI training Corpus
--------------------------------------------- Result 1490 ---------------------------------------------
Neutral (100%) --> Contradiction (49%)

Premise: Many women trying out and looking at laptops in a Walmart.
Hypothesis: Women are shopping for Dell Computers.

Premise: Many women trying out and looking at laptops in a Walmart.
Hypothesis: charwoman comprise patronize per Dell Computers.


[Succeeded / Failed / Total] 938 / 399 / 1490:  74% 1490/2000 [02:17<00:47, 10.85it/s]Try loading SNLI training Corpus
--------------------------------------------- Result 1491 ---------------------------------------------
Contradiction (99%) --> [FAILED]

Premise: A man with a plaid shirt stands with his hand on hip

In [ ]:
get_attack_freq_stats('../TrainedBertModels/roberta-base-snli/attack/freq_attacks_snli_FrequencyP_2000_attacked.csv',snli_freq_dict)

MEAN:	10.164118101802039	 STD:	2.7852977263496927
MEAN:	10.186937029004115	 STD:	2.745400451830755
MEAN:	0.10736771319771883	 STD:	0.5955133338700614
MEAN:	2.1961033927878817	 STD:	1.6329916524489152


In [ ]:
!ls ../TrainedBertModels/roberta-base-snli

1598279886.7127306				   pytorch_model.bin
attack						   README.md
config.json					   special_tokens_map.json
events.out.tfevents.1598279886.3762d14d0a9a.368.0  tokenizer_config.json
log.txt						   train_args.json
merges.txt					   vocab.json


# Defense Models

In [16]:
import argparse
import copy
import importlib
import json
import os
import time
import math 

import textattack

from textattack.commands.attack.attack_args import (
    ATTACK_RECIPE_NAMES,
    BLACK_BOX_TRANSFORMATION_CLASS_NAMES,
    CONSTRAINT_CLASS_NAMES,
    GOAL_FUNCTION_CLASS_NAMES,
    HUGGINGFACE_DATASET_BY_MODEL,
    SEARCH_METHOD_CLASS_NAMES,
    TEXTATTACK_DATASET_BY_MODEL,
    WHITE_BOX_TRANSFORMATION_CLASS_NAMES,
)

# The split token allows users to optionally pass multiple arguments in a single
# parameter by separating them with the split token.
ARGS_SPLIT_TOKEN = "^"

def _cb(s):
  return textattack.shared.utils.color_text(str(s), color="blue", method="ansi")

def phi_value(freq):
    return math.log(1+freq)

def load_module_from_file(file_path):
    """Uses ``importlib`` to dynamically open a file and load an object from
    it."""
    temp_module_name = f"temp_{time.time()}"
    colored_file_path = textattack.shared.utils.color_text(
        file_path, color="blue", method="ansi"
    )
    textattack.shared.logger.info(f"Loading module from `{colored_file_path}`.")
    spec = importlib.util.spec_from_file_location(temp_module_name, file_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

def parse_model(model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    if model_from_file:
        # Support loading the model from a .py file where a model wrapper
        # is instantiated.
        colored_model_name = textattack.shared.utils.color_text(
            model_from_file, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading model and tokenizer from file: {colored_model_name}"
        )
        if ARGS_SPLIT_TOKEN in model_from_file:
            model_file, model_name = model_from_file.split(ARGS_SPLIT_TOKEN)
        else:
            _, model_name = model_from_file, "model"
        try:
            model_module = load_module_from_file(model_from_file)
            print("))))HERE(((())))")
            print(model_module)
        except Exception:
            raise ValueError(f"Failed to import file {model_from_file}")
        try:
            model = getattr(model_module, model_name)
        except AttributeError:
            raise AttributeError(
                f"``{model_name}`` not found in module {model_from_file}"
            )

        if not isinstance(model, textattack.models.wrappers.ModelWrapper):
            
            if isinstance(model, textattack.models.NLINet):
                
                model = textattack.models.wrappers.PyTorchModelWrapper(
                    model, model.encoder, batch_size=model_batch_size
                )
                
            else:
                raise TypeError(
                    "Model must be of type "
                    f"``textattack.models.ModelWrapper``, got type {type(model)}"
                )

          
    elif (model in HUGGINGFACE_DATASET_BY_MODEL) or model_from_huggingface:
        # Support loading models automatically from the HuggingFace model hub.
        import transformers

        model_name = (
            HUGGINGFACE_DATASET_BY_MODEL[args.model][0]
            if (model in HUGGINGFACE_DATASET_BY_MODEL)
            else model_from_huggingface
        )

        if ARGS_SPLIT_TOKEN in model_name:
            model_class, model_name = model_name
            model_class = eval(f"transformers.{model_class}")
        else:
            model_class, model_name = (
                transformers.AutoModelForSequenceClassification,
                model_name,
            )
        colored_model_name = textattack.shared.utils.color_text(
            model_name, color="blue", method="ansi"
        )
        textattack.shared.logger.info(
            f"Loading pre-trained model from HuggingFace model repository: {colored_model_name}"
        )
        model = model_class.from_pretrained(model_name)
        tokenizer = textattack.models.tokenizers.AutoTokenizer(model_name)
        model = textattack.models.wrappers.HuggingFaceModelWrapper(
            model, tokenizer, batch_size=model_batch_size
        )
    elif model in TEXTATTACK_DATASET_BY_MODEL:
        # Support loading TextAttack pre-trained models via just a keyword.
        model_path, _ = TEXTATTACK_DATASET_BY_MODEL[model]
        model = textattack.shared.utils.load_textattack_model_from_path(
            model, model_path
        )
        # Choose the approprate model wrapper (based on whether or not this is
        # a HuggingFace model).
        if isinstance(
            model, textattack.models.helpers.BERTForClassification
        ) or isinstance(model, textattack.models.helpers.T5ForTextToText):
            model = textattack.models.wrappers.HuggingFaceModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
        else:
            model = textattack.models.wrappers.PyTorchModelWrapper(
                model, model.tokenizer, batch_size=model_batch_size
            )
    else:
      raise ValueError("Nothing else supported")
    

    return model

def parse_defense_from_class(defense_from_class=None,freq_dict=None,delta=10,type='discrete',gamma='0.1',candidates=8,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

    model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)
    
    if defense_from_class:
      import sys
      path = defense_from_class[0]
      path_class = path.rsplit('/')
      folder_path =  path_class[0] + '/'
      file = path_class[1]

      sys.path.insert(0,folder_path)
      module = importlib.import_module(file)
      my_class = getattr(module, defense_from_class[1])

      return my_class.build(model,freq_dict,delta=delta,type=type,gamma=gamma,candidates=candidates)
    else:
      raise ValueError("currently only attack from class is supported")

In [17]:
import sys
sys.path.insert(0,"../")

In [18]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack

logger = textattack.shared.logger

column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack,logger=True):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  #pbar = tqdm.autonotebook.tqdm(total=num_remaining_attacks)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  probabilities = []
  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    #or_label = result.original_result.output
    #initial_score = result.original_result.raw_output[or_label]
    #pertubated_score = result.perturbed_result.raw_output[or_label]
    initial_score = 1-result.original_result.score
    pertubated_score = 1-result.perturbed_result.score
    probabilities.append(initial_score-pertubated_score)

    if logger:
      attack_log_manager.log_result(result)

      print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )
    
  pbar.close()

  # Enable summary stdout
  
  if logger:
    attack_log_manager.log_summary()
    attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures, probabilities



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "snli",
      candidates = 8,
      logger = True,
      only_attack = False,
      only_original = False,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'modified_indices' , "attacked_label"])
    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"])
    or_labels = original_data["original_label"]

    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]' or or_labels[index]==x['attacked_label']:
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    attacked_data = attacked_data.drop(['modified_indices'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]


    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    if not only_original:
      true_positives, false_negatives, pert_probs = evaluate_set(attacked_dataset,attack,logger)
      self.tp = true_positives
      self.fn = false_negatives
      self.adv_probs = pert_probs
      print("True positives :" + str(true_positives) + "\n")
      print("False_negatives :" + str(false_negatives) + "\n")

    if  not only_attack:
      print("------Original Dataset------")
      false_positives, true_negatives, or_probs = evaluate_set(original_dataset,attack,logger)
      print("False positives :" + str(false_positives) + "\n")
      print("True negative :" + str(true_negatives) + "\n")
      self.fp = false_positives
      self.tn = true_negatives
      self.or_probs = or_probs

# Attack whole eval with random

In [19]:
def return_multi_nli_eval_dict(dataset):

  train_data = nlp.load_dataset(dataset)['train']

  valid_dict = {'premise':[],'hypothesis':[]}

  perc_ = math.ceil(len(train_data)*0.15)
  print("----EVAL DATASET USING : " + str(perc_) + "\n")

  eval_indices = np.sort(np.random.RandomState(21).choice(np.arange(len(train_data)),size=perc_,))

  valid_dict['premise'] = np.array(train_data['premise'])[eval_indices] 
  valid_dict['hypothesis'] = np.array(train_data['hypothesis'])[eval_indices]
  valid_dict['label'] = np.array(train_data['label'])[eval_indices]


  return valid_dict

In [20]:
# Create Eval datasets
snli_dataset = nlp.load_dataset("snli")

#multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
snli_eval_dataset = snli_dataset['validation']
snli_eval_dataset = {'premise':snli_eval_dataset['premise'],'hypothesis':snli_eval_dataset['hypothesis'],'label':snli_eval_dataset['label']}

In [21]:
from textattack.shared.utils import get_modifiable_indices

def parse_attack_from_recipe(recipe=None,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):

  model = parse_model(model_from_file, model,model_from_huggingface,model_batch_size)

  recipe = eval(f"{ATTACK_RECIPE_NAMES[recipe]}.build(model)")
  recipe.goal_function.query_budget = float("inf")
  recipe.goal_function.model_batch_size = model_batch_size
  recipe.goal_function.model_cache_size = 2 ** 18
  recipe.constraint_cache_size = 2 ** 18
  return recipe
    
def create_snli_dict():
    dictionary = {}
    dictionary["premise"] = []
    dictionary["original_hypothesis"] = []
    dictionary["attacked_hypothesis"] = []
    dictionary["label"] = []
    dictionary["original_label"] = []
    dictionary["attacked_label"] = []
    dictionary['modifiable_indices'] = []
    dictionary['modified_indices'] = []

    return dictionary

def create_csv_file(dictionary,filename):
    df = pd.DataFrame(dictionary)
    df.to_csv(filename, encoding='utf-8', index=False)
    return df

def attack_recipe(recipe,dataset,data_name,folder,model_from_file=None, model=None,model_from_huggingface=None,model_batch_size=32):
     
  # Get attack
  attack = parse_attack_from_recipe(recipe,model_from_file, model,model_from_huggingface,model_batch_size)
    
  #Remove untified instances
  to_remove = []
  for index,x in dataset.iterrows():
    if x['label'] == -1:
      to_remove.append(index)

  dataset = dataset.drop(to_remove)
  dataset = nlp.Dataset.from_pandas(dataset)

  dataset = textattack.datasets.HuggingFaceNlpDataset(
          name=data_name, shuffle=False, split=None, dataset=dataset, label_map = reversed_encoder)
    
  num_remaining_attacks = len(dataset)
  worklist = deque(range(0, len(dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0

  load_time = time.time()

  output_attack_dir = os.path.join(folder, "attack")
  output_attack_dir = os.path.normpath(output_attack_dir)
  if not os.path.exists(output_attack_dir):
    os.makedirs(output_attack_dir)

  print("\nAttacks will be saved at " + output_attack_dir + "\n")
  attacked_dict = create_snli_dict()

  for result in attack.attack_dataset(dataset, indices=worklist):

    modifiable_indices = get_modifiable_indices(result.original_result.attacked_text,attack.pre_transformation_constraints)
    prem = result.original_result.attacked_text._text_input['premise']
    hypothesis = result.original_result.attacked_text._text_input['hypothesis']
    attacked_dict['premise'].append(prem)
    attacked_dict['original_hypothesis'].append(hypothesis)
    result.change_word_numbers = len(hypothesis.split())
    attacked_dict['label'].append(result.original_result.ground_truth_output)
    attacked_dict['original_label'].append(result.original_result._processed_output[0])
    attacked_dict['modifiable_indices'].append(list(modifiable_indices))

    if isinstance(result, textattack.attack_results.SkippedAttackResult):
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.original_result._processed_output[0])
        attacked_dict['modified_indices'].append([])

    else:

        hypothesis = result.perturbed_result.attacked_text._text_input['hypothesis']
        attacked_dict['attacked_hypothesis'].append(hypothesis)
        attacked_dict['attacked_label'].append(result.perturbed_result._processed_output[0])
        attacked_dict['modified_indices'].append(list(result.perturbed_result.attacked_text.attack_attrs['modified_indices']))

   
    attack_log_manager.log_result(result)

    print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )

  pbar.close()

  # Enable summary stdout
  
  attack_log_manager.log_summary()
  attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  filename = output_attack_dir + "/" + recipe + "eval_whole" 

  create_csv_file(attacked_dict,filename+"_attacked.csv")

In [ ]:
#multi_eval_dataset = pd.DataFrame.from_dict(multi_eval_dataset)
snli_eval_dataset = pd.DataFrame.from_dict(snli_eval_dataset)
classifier = '../TrainedBertModels/roberta-base-snli'
attack_recipe("random_nli",snli_eval_dataset,"snli",classifier,model_from_file=None, model=None,model_from_huggingface=classifier,model_batch_size=32)


Output hidden; open in https://colab.research.google.com to view.

#Get prectiles for delta tuning

In [22]:
snli_dataset = nlp.load_dataset("snli")
#multi_dataset = nlp.load_dataset("multi_nli")
#multi_train_dataset = return_multi_nli_train_dict("multi_nli")
#multi_eval_dataset = return_multi_nli_eval_dict("multi_nli")
percentiles = [0,10,20,30,40,50,60,70,80,90,100]

In [23]:
# SNLI 
frequencies_train = []
for sentence in snli_dataset['train']['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,snli_freq_dict)
  frequencies_train += phi_values

print("SNLI frequency precentiles")
snli_precentiles = np.percentile(frequencies_train,percentiles)
print(snli_precentiles)

SNLI frequency precentiles
[ 0.69314718  6.3526294   7.83636976  8.92532142  9.80477158 10.72827684
 11.64400636 12.20202446 12.82231448 13.15694643 13.73591308]


In [ ]:
# MNLI
''' 
frequencies_train = []
for sentence in multi_train_dataset['hypothesis']:
  words = words_from_text(sentence)
  phi_values = get_phi_values(words,multi_freq_dict)
  frequencies_train += phi_values

print("MNLI frequency precentiles")
mnli_precentiles = np.percentile(frequencies_train,percentiles)
print(mnli_precentiles)
'''

' \nfrequencies_train = []\nfor sentence in multi_train_dataset[\'hypothesis\']:\n  words = words_from_text(sentence)\n  phi_values = get_phi_values(words,multi_freq_dict)\n  frequencies_train += phi_values\n\nprint("MNLI frequency precentiles")\nmnli_precentiles = np.percentile(frequencies_train,percentiles)\nprint(mnli_precentiles)\n'

# Tune delta for discete

In [ ]:
!ls ../TrainedBertModels/roberta-base-snli/attack

alzantot_gpt2_2000_attacked.csv
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_2000_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_2000_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_2000_attacked.csv


In [24]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

def boostrap_auc(or_probs,adv_probs,times):

  np.random.seed(21)
  labels = np.concatenate((np.ones((len(adv_probs),)),np.zeros((len(adv_probs),))))
  au_scores = np.zeros(times)
  for i in range(times):
    or_sample_probs = np.random.choice(or_probs,len(adv_probs))
    probs = np.concatenate((np.array(adv_probs),np.array(or_sample_probs)))
    au_scores[i] = roc_auc_score(labels,probs)
  
  return np.mean(au_scores)

Try loading SNLI training Corpus


In [ ]:
tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []
auc_scores = []

for i in snli_precentiles:
  print(str(i) + " delta \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = i,
        type = 'discrete',
        gamma = 1,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False)
  
  tps_discrete.append(stat.tp)
  fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)
  
  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,1000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)
  
  

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli


0.6931471805599453 delta 



textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH discrete--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 5 / 1 / 6:   0%|          | 6/3078 [00:00<01:37, 31.61it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 2301 / 777 / 3078: 100%|██████████| 3078/3078 [02:00<00:00, 25.63it/s]
textattack: Attack time: 120.11593174934387s
[Succeeded / Failed / Total] 2 / 2 / 4:   0%|          | 4/9842 [00:00<08:04, 20.30it/s]


True positives :2301

False_negatives :777

------Original Dataset------


[Succeeded / Failed / Total] 1860 / 7982 / 9842: 100%|██████████| 9842/9842 [06:29<00:00, 25.24it/s]
textattack: Attack time: 389.95661306381226s



False positives :1860

True negative :7982

F1 score 0.8582334283103059

AUC score 0.7281671234918331



In [ ]:
print("TPS")
print(tps_discrete)
print("\nFNS")
print(fns_discrete)
print("\nFPS")
print(fps_discrete)
print("\nTNS")
print(tns_discrete)
print("\nF1scores")
print(f1_discrete)
print("\nAUCscores")
print(auc_scores)

TPS
[1643, 2279, 2317, 2356, 2347, 2331, 2308, 2306, 2306, 2301, 2301]

FNS
[1435, 799, 761, 722, 731, 747, 770, 772, 772, 777, 777]

FPS
[15, 531, 909, 1209, 1462, 1618, 1820, 1829, 1845, 1863, 1860]

TNS
[9827, 9311, 8933, 8633, 8380, 8224, 8022, 8013, 7997, 7979, 7982]

F1scores
[0.9312926459438968, 0.9333400160384925, 0.9145167895167895, 0.8994113663593268, 0.8842927241070017, 0.8742890554403869, 0.8610067618332081, 0.8603639877597037, 0.859384235129762, 0.8580492526078074, 0.8582334283103059]

AUCscores
[0.7600446458992763, 0.824490599777245, 0.8028424936384352, 0.7859620467266282, 0.7641183037853579, 0.7497570594687571, 0.733164754080711, 0.7327253584620951, 0.7296015869185876, 0.7279579754623244, 0.7281671234918331]


In [ ]:
discrete_fp_rates = [fp / (fp+tn)  for fp,tn in zip(fps_discrete,tns_discrete)]
print("FPrate")
print(discrete_fp_rates)

discrete_tp_rates = [tp / (tp+fn)  for tp,fn in zip(tps_discrete,fns_discrete)]
print("\nTPrate")
print(discrete_tp_rates)
discrete_max_tp = np.argmax(discrete_tp_rates)
print("Maximum Tprate at delta :")
print(discrete_max_tp)
print("Best delta Value : ")
discrete_delta = snli_precentiles[discrete_max_tp]
print(discrete_delta)

FPrate
[0.0015240804714488924, 0.05395244868929079, 0.09235927656980289, 0.12284088599878074, 0.14854704328388538, 0.16439748018695388, 0.18492176386913228, 0.18583621215200163, 0.18746189798821378, 0.18929079455395245, 0.18898597845966267]

TPrate
[0.5337881741390513, 0.7404158544509422, 0.7527615334632879, 0.7654320987654321, 0.762508122157245, 0.7573099415204678, 0.7498375568551007, 0.7491877842755036, 0.7491877842755036, 0.7475633528265108, 0.7475633528265108]
Maximum Tprate at delta :
3
Best delta Value : 
8.925321416943886


# Tune gamma (this will be done manually) take the best delta and try values for gamma then check the fpr

In [25]:
discrete_delta = 8.92532142

In [26]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0,0.2,0.4,0.6,0.8,1]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli


Try loading SNLI training Corpus
0 gamma 



textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
  0%|          | 0/9842 [00:00<?, ?it/s]

...

--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 0 / 5 / 5:   0%|          | 5/9842 [00:00<05:55, 27.68it/s]

------Attacked Dataset------
------Original Dataset------


[Succeeded / Failed / Total] 0 / 9842 / 9842: 100%|██████████| 9842/9842 [04:24<00:00, 37.22it/s]
textattack: Attack time: 264.4615490436554s



False positives :0

True negative :9842



In [27]:
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score

print('Try loading SNLI training Corpus')
snli_freq_dict = corpus.load_csv('../snli_train_word_freq.csv')

tps_discrete = []
fns_discrete = []
fps_discrete = []
tns_discrete = []
f1_discrete = []



for i in [0.45,0.5,0.55,0.85,0.9,0.95]:
  print(str(i) + " gamma \n")

  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = i,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/random_nlieval_whole_attacked.csv',
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=True)
  
  #tps_discrete.append(stat.tp)
  #fns_discrete.append(stat.fn)
  fps_discrete.append(stat.fp)
  tns_discrete.append(stat.tn)

  #true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  #predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  #f1 = f1_score(true_labels, predictions)
  #print("F1 score " + str(f1) + '\n')
  #f1_discrete.append(f1)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli


Try loading SNLI training Corpus
0.45 gamma 



textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 1 / 5 / 6:   0%|          | 6/9842 [00:00<05:20, 30.66it/s]

...

--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 0 / 6 / 6:   0%|          | 6/9842 [00:00<05:27, 30.06it/s]

------Attacked Dataset------
------Original Dataset------


[Succeeded / Failed / Total] 344 / 9498 / 9842: 100%|██████████| 9842/9842 [04:23<00:00, 37.39it/s]
textattack: Attack time: 263.21008110046387s



False positives :344

True negative :9498



In [29]:
gamma_5 = 0.875
gamma_10 = 0.575 

# Defened against all attacks

In [31]:
!ls ../TrainedBertModels/roberta-base-snli/attack

alzantot_gpt2_2000_attacked.csv
freq_attacks_snli_FrequencyP_2000_attacked.csv
freq_attacks_snli_FrequencyR_2000_attacked.csv
prioritized_Priotirized_2000_attacked.csv
pwws_snli_2000_attacked.csv
random_nli_2000_attacked.csv
random_nlieval_whole_attacked.csv
textfooler_2000_attacked.csv


In [32]:
attacks = ["random_nli_2000_attacked.csv","prioritized_Priotirized_2000_attacked.csv","textfooler_2000_attacked.csv","pwws_snli_2000_attacked.csv","alzantot_gpt2_2000_attacked.csv"]

### Discrete

In [ ]:
import tqdm
pbar = tqdm.tqdm(total=10000, smoothing=0)
for i in range(10000):
  pbar.update(1)
pbar.close()


100%|██████████| 10000/10000 [00:00<00:00, 1227481.42it/s]


In [33]:
tpr_discrete = []
fpr_discrete = []
f1_discrete = []
auc_scores = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_discrete.append(stat.tp/(stat.tp+stat.fn))
  fpr_discrete.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_discrete.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_scores.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH discrete--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 5 / 0 / 5:   1%|          | 5/616 [00:00<00:22, 26.96it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 888 / 445 / 1333: 100%|██████████| 1333/1333 [00:41<00:00, 31.95it/s]
textattack: Attack time: 41.72303318977356s
[Succeeded / Failed / Total] 0 / 7 / 7:   0%|          | 7/2000 [00:00<00:50, 39.62it/s]


True positives :888

False_negatives :445

------Original Dataset------


[Succeeded / Failed / Total] 280 / 1720 / 2000: 100%|██████████| 2000/2000 [00:53<00:00, 37.48it/s]
textattack: Attack time: 53.3586163520813s



False positives :280

True negative :1720

F1 score 0.8259303721488596

AUC score 0.7776835084521317



In [34]:
print("TPrate")
print(tpr_discrete)
print("\nFPrate")
print(fpr_discrete)
print("\nF1 score")
print(f1_discrete)
print("\n AUC scores")
print(auc_scores)

TPrate
[0.737012987012987, 0.7595854922279792, 0.6612623045744065, 0.7311291328758578, 0.6661665416354089]

FPrate
[0.14, 0.14, 0.14, 0.14, 0.14]

F1 score
[0.8861411643482741, 0.8704453441295547, 0.7990708478513355, 0.8287159720549265, 0.8259303721488596]

 AUC scores
[0.7628390477420307, 0.8035530558135789, 0.7350902043031267, 0.8003836481737105, 0.7776835084521317]


### Continuous 10%

In [35]:
tpr_cont10 = []
fpr_cont10 = []
f1_cont10 = []
auc_cont10 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_10,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont10.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont10.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont10.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont10.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 3 / 2 / 5:   1%|          | 5/616 [00:00<00:22, 27.03it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 694 / 639 / 1333: 100%|██████████| 1333/1333 [00:42<00:00, 31.66it/s]
textattack: Attack time: 42.11083507537842s
[Succeeded / Failed / Total] 0 / 7 / 7:   0%|          | 7/2000 [00:00<00:51, 38.82it/s]


True positives :694

False_negatives :639

------Original Dataset------


[Succeeded / Failed / Total] 234 / 1766 / 2000: 100%|██████████| 2000/2000 [00:54<00:00, 36.75it/s]
textattack: Attack time: 54.42431879043579s



False positives :234

True negative :1766

F1 score 0.801816118047673

AUC score 0.7776835084521317



In [36]:
print("TPrate")
print(tpr_cont10)
print("\nFPrate")
print(fpr_cont10)
print("\nF1 score")
print(f1_cont10)
print("\n AUC scores")
print(auc_cont10)

TPrate
[0.45616883116883117, 0.6113989637305699, 0.4400694846554719, 0.6269494697442296, 0.5206301575393848]

FPrate
[0.117, 0.117, 0.117, 0.117, 0.117]

F1 score
[0.8612533528407706, 0.8529340738951944, 0.7462497358968941, 0.8093492208982586, 0.801816118047673]

 AUC scores
[0.7628390477420307, 0.8035530558135789, 0.7350902043031267, 0.8003836481737105, 0.7776835084521317]


## Continuous 5%

In [37]:
tpr_cont5 = []
fpr_cont5 = []
f1_cont5 = []
auc_cont5 = []


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'continuous',
        gamma = gamma_5,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_cont5.append(stat.tp/(stat.tp+stat.fn))
  fpr_cont5.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_cont5.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_cont5.append(auc_score)

textattack: Loading pre-trained model from HuggingFace model repository: ../TrainedBertModels/roberta-base-snli


Attack From
random_nli_2000_attacked.csv




textattack: Unknown if model of class <class 'textattack.models.wrappers.huggingface_model_wrapper.HuggingFaceModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


--Detection SEARCH continuous--


textattack: Loading nlp dataset snli, split None.
textattack: Loading nlp dataset snli, split None.
[Succeeded / Failed / Total] 1 / 4 / 5:   1%|          | 5/616 [00:00<00:23, 26.47it/s]

...

------Attacked Dataset------


[Succeeded / Failed / Total] 353 / 980 / 1333: 100%|██████████| 1333/1333 [00:42<00:00, 31.66it/s]
textattack: Attack time: 42.104472160339355s
[Succeeded / Failed / Total] 0 / 7 / 7:   0%|          | 7/2000 [00:00<00:51, 38.91it/s]


True positives :353

False_negatives :980

------Original Dataset------


[Succeeded / Failed / Total] 118 / 1882 / 2000: 100%|██████████| 2000/2000 [00:53<00:00, 37.48it/s]
textattack: Attack time: 53.36493802070618s



False positives :118

True negative :1882

F1 score 0.774167009461127

AUC score 0.7776835084521317



In [38]:
print("TPrate")
print(tpr_cont5)
print("\nFPrate")
print(fpr_cont5)
print("\nF1 score")
print(f1_cont5)
print("\n AUC scores")
print(auc_cont5)

TPrate
[0.1590909090909091, 0.29015544041450775, 0.1482339316734221, 0.3431066749844042, 0.26481620405101275]

FPrate
[0.059, 0.059, 0.059, 0.059, 0.059]

F1 score
[0.8554545454545456, 0.8241734179986862, 0.7031571081636466, 0.7627152988855115, 0.774167009461127]

 AUC scores
[0.7628390477420307, 0.8035530558135789, 0.7350902043031267, 0.8003836481737105, 0.7776835084521317]


# NWS and FGWS

In [39]:
import scipy
import torch
import pandas as pd
from collections import deque
import corpus
import tqdm
import time
import textattack
from sklearn.metrics import accuracy_score

logger = textattack.shared.logger

                               
column_encoder = {"Entailment":0, "Neutral":1, "Contradiction":2}
reversed_encoder = {0:"Entailment", 1:"Neutral", 2:"Contradiction"}

def parse_log_manager():
    # Create logger
    attack_log_manager = textattack.loggers.AttackLogManager()

    '''
    # Get current time for file naming
    timestamp = time.strftime("%Y-%m-%d-%H-%M")

    # Get default directory to save results
    current_dir = os.path.dirname(os.path.realpath(__file__))
    outputs_dir = os.path.join(
        current_dir, os.pardir, os.pardir, os.pardir, "outputs", "attacks"
    )
    out_dir_txt = out_dir_csv = os.path.normpath(outputs_dir)

    # Get default txt and csv file names
    filename_txt = f"_{timestamp}.txt"
    filename_csv = f"_{timestamp}.csv"
    
    

    # in case directory doesn't exist
    if not os.path.exists(out_dir_txt):
        os.makedirs(out_dir_txt)
    if not os.path.exists(out_dir_csv):
        os.makedirs(out_dir_csv)

    attack_log_manager.add_output_file(os.path.join(out_dir_txt, filename_txt))
    '''
    #attack_log_manager.enable_visdom()
    #attack_log_manager.enable_wandb()
    attack_log_manager.enable_stdout()
    return attack_log_manager

def evaluate_set(attacked_dataset,attack,correct_labels,logger=True):

  num_remaining_attacks = len(attacked_dataset)
  worklist = deque(range(0, len(attacked_dataset)))
  start_time = time.time()
  worklist_tail = worklist[-1]

  pbar = tqdm.tqdm(total=num_remaining_attacks, smoothing=0)
  #pbar = tqdm.autonotebook.tqdm(total=num_remaining_attacks)
  attack_log_manager = parse_log_manager()


  num_results = 0
  num_failures = 0
  num_successes = 0
  transform_labels = []

  load_time = time.time()

  probabilities = []
  for result in attack.attack_dataset(attacked_dataset, indices=worklist):

    #or_label = result.original_result.output
    #initial_score = result.original_result.raw_output[or_label]
    #pertubated_score = result.perturbed_result.raw_output[or_label]
    initial_score = 1-result.original_result.score
    pertubated_score = 1-result.perturbed_result.score
    probabilities.append(initial_score-pertubated_score)
    transform_labels.append(result.perturbed_result.output)

    if logger:
      attack_log_manager.log_result(result)

      print("\n")
    pbar.update(1)
    
    num_results += 1

    if (
        type(result) == textattack.attack_results.SuccessfulAttackResult
        or type(result) == textattack.attack_results.MaximizedAttackResult
    ):
        num_successes += 1
    if type(result) == textattack.attack_results.FailedAttackResult:
        num_failures += 1
    
    pbar.set_description(
        "[Succeeded / Failed / Total] {} / {} / {}".format(
            num_successes, num_failures, num_results
        )
    )
    
  pbar.close()

  # Enable summary stdout
  
  if logger:
    attack_log_manager.log_summary()
    attack_log_manager.flush()
  print()
  textattack.shared.logger.info(f"Attack time: {time.time() - load_time}s")

  return num_successes, num_failures, probabilities, transform_labels



class EvalDefenseModel():

  def __init__(
      self,
      defense_from_class,
      model_from_file=None, 
      model=None,
      model_from_huggingface=None,
      model_batch_size=32,
      delta = 10,
      gamma = 1,
      type = 'discrete',
      freq_dict = None,
      data_file = None,
      data_name = "snli",
      candidates = 8,
      logger = True,
      only_attack = False,
      only_original = False,
      #indices_to_modify=None,
      #shifted_idxs=True,
    ):

    # Get attack
    attack = parse_defense_from_class(defense_from_class,freq_dict,delta,type,gamma,candidates,model_from_file, model,model_from_huggingface,model_batch_size)
    
    #Get datasets
    attacked_data = pd.read_csv(data_file,usecols=["premise",'attacked_hypothesis', 'label', 'modified_indices' , "attacked_label"])
    original_data = pd.read_csv(data_file,usecols=["premise",'original_hypothesis', "original_label"])
    or_labels = original_data["original_label"]
    

    to_remove = []
    for index,x in attacked_data.iterrows():
      if x['modified_indices'] == '[]':
        to_remove.append(index)
    attacked_data = attacked_data.drop(to_remove)
    correct_labels = attacked_data['label']
    
    

    attacked_data = attacked_data.drop(['modified_indices','label'],axis=1)

    attacked_data["attacked_label"] = [column_encoder[i] for i in attacked_data["attacked_label"] ]
    # Get original adv accuracy 
    self.adv_acc = accuracy_score(correct_labels,attacked_data["attacked_label"])

    original_data["original_label"] = [column_encoder[i] for i in original_data["original_label"] ]

    attacked_data = attacked_data.rename(columns={'attacked_hypothesis': 'hypothesis','attacked_label':'label'})
    attacked_dataset = nlp.Dataset.from_pandas(attacked_data)

    attacked_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=attacked_dataset, label_map = reversed_encoder)
    
    original_data = original_data.rename(columns={'original_hypothesis': 'hypothesis','original_label':'label'})
    original_dataset = nlp.Dataset.from_pandas(original_data)

    original_dataset = textattack.datasets.HuggingFaceNlpDataset(
            name=data_name, shuffle=False, split=None, dataset=original_dataset, label_map = reversed_encoder)
    
    print("------Attacked Dataset------")
    if not only_original:
      true_positives, false_negatives, pert_probs, transform_labels = evaluate_set(attacked_dataset,attack,logger)
      self.tp = true_positives
      self.fn = false_negatives
      self.adv_probs = pert_probs

      print("True positives :" + str(true_positives) + "\n")
      print("False_negatives :" + str(false_negatives) + "\n")

      # Get transformed adv accuracy 
      self.transform_acc = accuracy_score(correct_labels,transform_labels)

    if  not only_attack:
      print("------Original Dataset------")
      false_positives, true_negatives, or_probs, _i10 = evaluate_set(original_dataset,attack,logger)
      print("False positives :" + str(false_positives) + "\n")
      print("True negative :" + str(true_negatives) + "\n")
      self.fp = false_positives
      self.tn = true_negatives
      self.or_probs = or_probs

In [40]:
tpr_nws = []
fpr_nws = []
f1_nws = []
auc_nws = []
adv_acc_nws = []
tr_acc_nws =[]


for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "NWS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = None,
        type = None,
        gamma = None,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_nws.append(stat.tp/(stat.tp+stat.fn))
  fpr_nws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_nws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_nws.append(auc_score)

  adv_acc_nws.append(stat.adv_acc) 
  tr_acc_nws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))


Output hidden; open in https://colab.research.google.com to view.

In [41]:
print("TP rates")
print(tpr_nws)
print("\nFP rates")
print(fpr_nws)
print("\nF1 scores")
print(f1_nws)
print("\nAuc scores")
print(auc_nws)
print("\nAdv_acc")
print(adv_acc_nws)
print("\nTransform_acc")
print(tr_acc_nws)

TP rates
[0.1449763991908294, 0.20900774102744546, 0.2155124653739612, 0.19911259012756516, 0.1735261401557286]

FP rates
[0.002, 0.0015, 0.001, 0.002, 0.0035]

F1 scores
[0.7583586626139819, 0.779925795743019, 0.7380864425563354, 0.7338235294117647, 0.7275050191640811]

Auc scores
[0.5430283947675817, 0.5647086070459146, 0.5983693409657691, 0.5895397783751675, 0.5730734137300003]

Adv_acc
[0.5846257585974376, 0.32090077410274453, 0.043213296398891966, 0.11092623405435385, 0.25862068965517243]

Transform_acc
[0.6824005394470668, 0.5151301900070373, 0.24487534626038782, 0.29395452024403773, 0.4082313681868743]


In [44]:
!ls ../TrainedBertModels/roberta-base-snli/

1598279886.7127306				   pytorch_model.bin
attack						   README.md
config.json					   special_tokens_map.json
events.out.tfevents.1598279886.3762d14d0a9a.368.0  tokenizer_config.json
log.txt						   train_args.json
merges.txt					   vocab.json


In [45]:
tpr_fgws = []
fpr_fgws = []
f1_fgws = []
auc_fgws = []
adv_acc_fgws = []
tr_acc_fgws = []

for attack in attacks:
  print("Attack From")
  print(attack)
  print("\n")
  stat = EvalDefenseModel(defense_from_class=["../detect_adversarial", "FWGS"],
        model_from_file=None, 
        model=None,
        model_from_huggingface='../TrainedBertModels/roberta-base-snli',
        model_batch_size=32,
        delta = discrete_delta,
        type = 'discrete',
        gamma = None,
        freq_dict = snli_freq_dict,
        data_file='../TrainedBertModels/roberta-base-snli/attack/' + attack,
        data_name = 'snli',
        candidates =8,
        logger = False,
        only_attack = False,
        only_original=False)
    
  tpr_fgws.append(stat.tp/(stat.tp+stat.fn))
  fpr_fgws.append(stat.fp/(stat.fp+stat.tn))

  true_labels = np.concatenate((np.zeros(stat.tp+stat.fn),np.ones(stat.fp+stat.tn)))
  predictions = np.concatenate((np.zeros(stat.tp),np.ones(stat.fn),np.zeros(stat.fp),np.ones(stat.tn)))
  f1 = f1_score(true_labels, predictions)
  print("F1 score " + str(f1) + '\n')
  f1_fgws.append(f1)

  auc_score = boostrap_auc(stat.or_probs,stat.adv_probs,10000)
  print("AUC score " + str(auc_score) + '\n')
  auc_fgws.append(auc_score)
  
  adv_acc_fgws.append(stat.adv_acc) 
  tr_acc_fgws.append(stat.transform_acc)
  print("Adversarial Acc " + str(stat.adv_acc))
  print("Transform Acc " + str(stat.transform_acc))

Output hidden; open in https://colab.research.google.com to view.

In [46]:
print("TP rates")
print(tpr_fgws)
print("\nFP rates")
print(fpr_fgws)
print("\nF1 scores")
print(f1_fgws)
print("\nAuc scores")
print(auc_fgws)
print("\nAdv_acc")
print(adv_acc_fgws)
print("\nTransform_acc")
print(tr_acc_fgws)

TP rates
[0.3607552258934592, 0.5439831104855736, 0.6332409972299169, 0.653910149750416, 0.5088987764182424]

FP rates
[0.14, 0.14, 0.14, 0.14, 0.14]

F1 scores
[0.7369323050556984, 0.7875457875457875, 0.7850296668188042, 0.7918968692449355, 0.7473386921572888]

Auc scores
[0.5378933282756381, 0.6213582948494013, 0.7066545034031353, 0.7222882775949002, 0.6155399814990331]

Adv_acc
[0.5846257585974376, 0.32090077410274453, 0.043213296398891966, 0.11092623405435385, 0.25862068965517243]

Transform_acc
[0.8071476736345247, 0.7614356087262492, 0.6193905817174515, 0.7199112590127565, 0.6902113459399333]
